# Tutorial - Lagged Choices

The models in Keane and Wolpin (1994) and the extended model in Keane and Wolpin (1997) require information on the previous choice as some covariates rely on the information. In contrast, the base model in Keane and Wolpin (1997) does not require this information. This notebook shows how lags of choices can be included in your model.

## How to define lagged choices

Currently, the number of lags is inferred from the definition of covariates and the core state space filters. The lags have to be called ``lagged_choice_1``, ``lagged_choice_2``, and so on. The covariates are used in rewards, to calculate type probabilities and set initial conditions. The core state space filters reduce inadmissible states from the state space which are caused by combinations between experiences and lagged choices. Here are both options from the extended model in Keane and Wolpin (1997).

In [1]:
import respy as rp

In [5]:
params, options = rp.get_example_model("kw_97_extended", with_data=False)

The covariates are stored in a dictionary where the key matches parameter names from ``params`` and the value is the definition of the covariate. The information in the state space - period, experiences, lagged choices - are already available. ``exp_*`` is the code to target the experience in one choice. Lagged choices are defined internally as a [``pandas.Categorical``](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Categorical.html) and can be accessed by choice names. The choice names are inferred from ``params`` and have to be consistently used across parameters and options. It is possible to use previously defined covariates to define later ones.

In [6]:
options["covariates"]

{'not_exp_a_lagged': "exp_a > 0 and lagged_choice_1 != 'a'",
 'not_exp_b_lagged': "exp_b > 0 and lagged_choice_1 != 'b'",
 'work_a_lagged': "lagged_choice_1 == 'a'",
 'work_b_lagged': "lagged_choice_1 == 'b'",
 'edu_lagged': "lagged_choice_1 == 'edu'",
 'not_any_exp_a': 'exp_a == 0',
 'not_any_exp_b': 'exp_b == 0',
 'not_any_exp_mil': 'exp_mil == 0',
 'any_exp_a': 'exp_a > 0',
 'any_exp_b': 'exp_b > 0',
 'any_exp_mil': 'exp_mil > 0',
 'hs_graduate': 'exp_edu >= 12',
 'co_graduate': 'exp_edu >= 16',
 'is_return_not_high_school': '~edu_lagged and ~hs_graduate',
 'is_return_high_school': '~edu_lagged and hs_graduate',
 'is_minor': 'period < 2',
 'is_young_adult': '2 <= period <= 4',
 'is_adult': '5 <= period',
 'constant': '1',
 'exp_a_square': 'exp_a ** 2 / 100',
 'exp_b_square': 'exp_b ** 2 / 100',
 'exp_mil_square': 'exp_mil ** 2 / 100',
 'mil_dropout': 'exp_mil == 1',
 'up_to_nine_years_edu': 'exp_edu <= 9',
 'at_least_ten_years_edu': '10 <= exp_edu'}

Core state space filters remove impossible combinations from the state space. This reduces the model complexity and leads to better performance. They are called core state space filters because the state space does not include types or other dimensions of observable variables yet. The filters are defined as a list of strings with conditions. If the condition evaluates to ``True``, the state is removed.

In [7]:
options["core_state_space_filters"]

["period > 0 and exp_{i} == period and lagged_choice_1 != '{i}'",
 "period > 0 and exp_a + exp_b + exp_mil + exp_edu == period and lagged_choice_1 == '{j}'",
 "period > 0 and lagged_choice_1 == 'edu' and exp_edu == 0",
 "lagged_choice_1 == '{k}' and exp_{k} == 0",
 "period == 0 and lagged_choice_1 == '{k}'"]

## Lagged choices and model complexity



In [8]:
state_space = rp.solve(params, options)

MemoryError: 